# EGM/PEAD Stock Scanner

**Episodic Gap Momentum / Post-Earnings Announcement Drift Scanner**

This notebook provides an interactive interface for the EGM/PEAD stock scanning strategy.

## Strategy Overview

The PEAD (Post-Earnings Announcement Drift) anomaly shows that stocks continue drifting in the direction of an earnings surprise for ~60 days. The EGM strategy identifies:

1. **Episodic Pivots** - 10%+ price gaps on earnings/catalyst with massive volume
2. **Earnings Surprise** - >25% EPS beat vs consensus
3. **Consolidation Phase** - Stock builds a base after initial gap
4. **Entry Trigger** - Breakout from consolidation or pullback to key levels

---

## 1. Setup & Imports

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
plt.style.use('seaborn-v0_8-darkgrid')

# Scanner imports (from local package)
import sys
sys.path.insert(0, '..')

from Stock import (
    EGMScanner,
    GapDetector,
    EarningsValidator,
    TechnicalAnalyzer,
    IndustryClassifier,
    SetupAnalyzer,
    SignalGenerator,
    APIClient,
    BacktestEngine,
)

from Stock.signal_generator import format_signal_report, format_scan_summary

print("Imports complete!")
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

## 2. Configuration

In [ ]:
# API Keys (set your keys here or use environment variables)
# Leave as None to use yfinance only (no earnings calendar access)

FINNHUB_API_KEY = None  # Get free key at https://finnhub.io/
ALPHA_VANTAGE_KEY = None  # Get free key at https://www.alphavantage.co/

# Initialize scanner
scanner = EGMScanner(
    finnhub_key=FINNHUB_API_KEY,
    alpha_vantage_key=ALPHA_VANTAGE_KEY
)

print("Scanner initialized!")

---

## 3. Daily Scanner Dashboard

Scan stocks that reported earnings recently for EGM setups.

In [ ]:
# Scan stocks with recent earnings (last 5 days)
results, stats = scanner.scan_earnings_today(days_back=5, show_progress=True)

# Print results
scanner.print_results(results, stats)

In [ ]:
# Get actionable signals only (score >= 65)
actionable = scanner.get_actionable_signals(results, min_score=65)

if actionable:
    print(f"\n{len(actionable)} Actionable Signals:")
    print("-" * 50)
    for r in actionable:
        print(f"{r.ticker}: Score {r.signal.total_score:.0f} | {r.signal.recommendation.upper()}")
        if r.signal.entry_price:
            print(f"  Entry: ${r.signal.entry_price:.2f} | Stop: ${r.signal.stop_loss:.2f} | Target: ${r.signal.target_price:.2f}")
else:
    print("No actionable signals found.")

---

## 4. Custom Universe Scan

Scan a custom list of tickers.

In [ ]:
# Define your watchlist
WATCHLIST = [
    'NVDA', 'AMD', 'SMCI', 'CRWD', 'PANW',  # Tech
    'TSLA', 'RIVN', 'LCID',  # EVs
    'COIN', 'MARA', 'RIOT',  # Crypto
    'META', 'GOOGL', 'AMZN',  # Mega-cap tech
]

# Scan custom universe
results_custom, stats_custom = scanner.scan_universe(WATCHLIST, show_progress=True)

# Print results
scanner.print_results(results_custom, stats_custom)

---

## 5. Single Stock Analysis

Deep dive analysis on a specific ticker.

In [ ]:
# Analyze a single stock
TICKER = "NVDA"  # Change this to any ticker

result = scanner.analyze_stock(TICKER)

if result:
    # Print detailed signal report
    print(format_signal_report(result.signal))
    
    # Additional details
    if result.gap:
        print(f"\nGap Details:")
        print(f"  Date: {result.gap.gap_date}")
        print(f"  Gap: {result.gap.gap_percent:+.1f}%")
        print(f"  Volume: {result.gap.volume_multiple:.1f}x average")
        
    if result.earnings:
        print(f"\nEarnings:")
        print(f"  EPS Surprise: {result.earnings.eps_surprise_pct:.1f}%" if result.earnings.eps_surprise_pct else "  EPS: N/A")
        print(f"  Revenue Surprise: {result.earnings.revenue_surprise_pct:.1f}%" if result.earnings.revenue_surprise_pct else "  Revenue: N/A")
        
    if result.relative_strength:
        print(f"\nIndustry Relative Strength:")
        print(f"  Rank: {result.relative_strength.percentile_rank:.0f}th percentile")
        print(f"  Leader: {'Yes' if result.relative_strength.is_leader else 'No'}")
else:
    print(f"Could not analyze {TICKER}")

---

## 6. Gap Detection Visualization

In [ ]:
def plot_gap_chart(ticker, days=90):
    """
    Plot price chart with gap detection.
    """
    api = APIClient()
    df = api.fetch_price_history(ticker, days=days)
    
    if df is None or len(df) < 20:
        print(f"Not enough data for {ticker}")
        return
    
    # Detect gaps
    detector = GapDetector()
    gap_analysis = detector.analyze(df, ticker)
    
    # Create chart
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), height_ratios=[3, 1], sharex=True)
    
    # Price chart
    ax1.plot(df.index, df['close'], 'b-', linewidth=1.5, label='Close')
    ax1.fill_between(df.index, df['low'], df['high'], alpha=0.3, color='blue')
    
    # Mark gaps
    for gap in gap_analysis.gaps:
        if gap.meets_criteria:
            color = 'green' if gap.gap_direction == 'up' else 'red'
            ax1.axvline(gap.gap_date, color=color, linestyle='--', alpha=0.7, linewidth=2)
            ax1.annotate(f'{gap.gap_percent:+.1f}%',
                        xy=(gap.gap_date, gap.high_price),
                        xytext=(10, 10), textcoords='offset points',
                        fontsize=10, fontweight='bold', color=color)
    
    # Add moving averages
    if len(df) >= 50:
        ax1.plot(df.index, df['close'].rolling(10).mean(), 'orange', linewidth=1, label='10 SMA', alpha=0.7)
        ax1.plot(df.index, df['close'].rolling(20).mean(), 'purple', linewidth=1, label='20 SMA', alpha=0.7)
        ax1.plot(df.index, df['close'].rolling(50).mean(), 'gray', linewidth=1, label='50 SMA', alpha=0.7)
    
    ax1.set_title(f'{ticker} - Gap Detection ({len(gap_analysis.gaps)} gaps found)', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Price ($)')
    ax1.legend(loc='upper left')
    ax1.grid(True, alpha=0.3)
    
    # Volume chart
    avg_vol = df['volume'].rolling(20).mean()
    colors = ['green' if df['close'].iloc[i] >= df['open'].iloc[i] else 'red' for i in range(len(df))]
    ax2.bar(df.index, df['volume'], color=colors, alpha=0.7)
    ax2.plot(df.index, avg_vol, 'blue', linewidth=1.5, label='20-day Avg')
    ax2.set_ylabel('Volume')
    ax2.legend(loc='upper left')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print gap summary
    if gap_analysis.gaps:
        print(f"\nGap Summary for {ticker}:")
        print("-" * 50)
        for gap in gap_analysis.gaps:
            status = "QUALIFYING" if gap.meets_criteria else "Below threshold"
            print(f"  {gap.gap_date.strftime('%Y-%m-%d')}: {gap.gap_percent:+.1f}% ({gap.volume_multiple:.1f}x vol) - {status}")

# Example usage
plot_gap_chart('NVDA', days=120)

---

## 7. Technical Analysis View

In [ ]:
def analyze_technicals(ticker):
    """
    Display technical analysis for a ticker.
    """
    api = APIClient()
    df = api.fetch_price_history(ticker, days=90)
    
    if df is None:
        print(f"Could not fetch data for {ticker}")
        return
    
    analyzer = TechnicalAnalyzer()
    analysis = analyzer.analyze(df, ticker)
    
    print(f"\n{'='*50}")
    print(f"Technical Analysis: {ticker}")
    print(f"{'='*50}")
    
    print(f"\nPrice Information:")
    print(f"  Current Price:  ${analysis.current_price:.2f}")
    print(f"  10-day SMA:     ${analysis.sma_10:.2f}")
    print(f"  20-day SMA:     ${analysis.sma_20:.2f}")
    print(f"  50-day SMA:     ${analysis.sma_50:.2f}")
    print(f"  RSI(14):        {analysis.rsi_14:.1f}")
    
    print(f"\nMA Position:")
    print(f"  Above 10-SMA:   {'Yes' if analysis.above_10sma else 'No'}")
    print(f"  Above 20-SMA:   {'Yes' if analysis.above_20sma else 'No'}")
    print(f"  Above 50-SMA:   {'Yes' if analysis.above_50sma else 'No'}")
    
    print(f"\nVolatility Profile:")
    vol = analysis.volatility
    adr_status = 'PASS' if vol.adr_meets_criteria else 'FAIL'
    atr_status = 'PASS' if vol.atr_meets_criteria else 'FAIL'
    print(f"  ADR%:           {vol.adr_percent:.2f}% ({adr_status} - min 6%)")
    print(f"  ATR%:           {vol.atr_percent:.2f}% ({atr_status} - min 7%)")
    
    print(f"\nMA Cluster (Pre-Catalyst):")
    cluster = analysis.ma_cluster
    print(f"  Spread:         {cluster.spread_percent:.2f}%")
    print(f"  Quality:        {cluster.cluster_quality.upper()}")
    print(f"  Clustered:      {'Yes' if cluster.is_clustered else 'No'}")
    
    return analysis

# Example
_ = analyze_technicals('NVDA')

---

## 8. Industry Relative Strength

In [ ]:
def show_industry_comparison(ticker):
    """
    Show industry relative strength comparison.
    """
    classifier = IndustryClassifier()
    
    # Get industry info
    info = classifier.get_industry_info(ticker)
    
    print(f"\n{'='*50}")
    print(f"Industry Classification: {ticker}")
    print(f"{'='*50}")
    print(f"  Sector:      {info.sector}")
    print(f"  Industry:    {info.industry}")
    print(f"  Description: {info.description}")
    print(f"  Peers:       {info.peer_count} stocks")
    print(f"  Source:      {info.classification_source}")
    
    if info.peers:
        print(f"\n  Sample Peers: {', '.join(info.peers[:10])}")
        
        # Get peer comparison
        comparison = classifier.compare_to_peers(ticker)
        if comparison is not None:
            print(f"\nPeer Comparison (20-day returns):")
            print(comparison.to_string())
    
    return info

# Example
_ = show_industry_comparison('NVDA')

---

## 9. Strategy Backtest

In [ ]:
# Backtest configuration
BACKTEST_TICKERS = ['NVDA', 'AMD', 'AAPL', 'MSFT', 'GOOGL', 'META', 'AMZN']
LOOKBACK_DAYS = 365  # 1 year

# Fetch historical data
print("Fetching historical data...")
api = APIClient()
historical_data = {}

for ticker in BACKTEST_TICKERS:
    df = api.fetch_price_history(ticker, days=LOOKBACK_DAYS)
    if df is not None and len(df) > 50:
        historical_data[ticker] = df
        print(f"  {ticker}: {len(df)} days loaded")

print(f"\nLoaded data for {len(historical_data)} tickers")

In [ ]:
# Run backtest
if historical_data:
    engine = BacktestEngine(
        initial_capital=100_000,
        max_position_pct=10.0,
        max_positions=5
    )
    
    result = engine.run_backtest(
        historical_data,
        verbose=True
    )
else:
    print("No historical data available for backtest")

In [ ]:
# Plot equity curve
if historical_data and 'result' in dir() and len(result.equity_curve) > 0:
    fig, ax = plt.subplots(figsize=(12, 6))
    
    ax.plot(result.equity_curve.index, result.equity_curve['equity'], 'b-', linewidth=2)
    ax.axhline(y=100000, color='gray', linestyle='--', alpha=0.5, label='Starting Capital')
    
    ax.set_title('EGM/PEAD Strategy Equity Curve', fontsize=14, fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel('Portfolio Value ($)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No equity curve data available")

---

## 10. Export Results

In [ ]:
# Export scan results to CSV
if 'results' in dir() and results:
    export_df = scanner.export_results(results, 'egm_scan_results.csv')
    display(export_df.head(10))
else:
    print("No scan results to export")

---

## 11. Watchlist Tracking

In [ ]:
# Track active positions / watchlist
ACTIVE_WATCHLIST = [
    {'ticker': 'NVDA', 'entry': 120.00, 'stop': 108.00, 'target': 144.00},
    {'ticker': 'AMD', 'entry': 145.00, 'stop': 130.00, 'target': 174.00},
    # Add your positions here
]

def check_watchlist(watchlist):
    """
    Check current prices against watchlist entries.
    """
    api = APIClient()
    
    print("\n" + "="*70)
    print("WATCHLIST STATUS")
    print("="*70)
    print(f"{'Ticker':<8} {'Entry':>10} {'Current':>10} {'Stop':>10} {'Target':>10} {'P/L':>10} {'Status':>10}")
    print("-"*70)
    
    for item in watchlist:
        ticker = item['ticker']
        df = api.fetch_price_history(ticker, days=5)
        
        if df is not None and len(df) > 0:
            current = df['close'].iloc[-1]
            entry = item['entry']
            stop = item['stop']
            target = item['target']
            
            pnl = ((current / entry) - 1) * 100
            
            if current <= stop:
                status = 'STOP HIT'
            elif current >= target:
                status = 'TARGET'
            elif current > entry:
                status = 'PROFIT'
            else:
                status = 'LOSS'
            
            print(f"{ticker:<8} ${entry:>9.2f} ${current:>9.2f} ${stop:>9.2f} ${target:>9.2f} {pnl:>+9.1f}% {status:>10}")
    
    print("="*70)

if ACTIVE_WATCHLIST:
    check_watchlist(ACTIVE_WATCHLIST)

---

## Notes

### Entry Criteria Summary

| Criteria | Threshold | Weight |
|----------|-----------|--------|
| Gap Magnitude | >= 10% | 15% |
| Volume Spike | >= 3x avg | 15% |
| EPS Surprise | >= 25% | 20% |
| ADR% | >= 6% | 6% |
| ATR% | >= 7% | 6% |
| MA Cluster | <= 3% spread | 12% |
| Consolidation Quality | 2-10 days | 10% |
| Industry Relative Strength | Top 25% | 16% |

### Exit Rules

- **Partial Exit (50%)**: Price closes below 10-day SMA
- **Full Exit**: Price closes below 20-day SMA
- **Time Stop**: 60 days maximum hold
- **Profit Target**: 20% gain